# Introduction


This notebook demonstrates how BioThings Explorer can be used to execute queries having more than one intermediate nodes (here a "three-hop query").  **Note that this notebook can also be run and modified in Google CoLab ([link](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/Disease%20-%20Gene%20-%20Disease%20-%20Chem%202021-02-03%20relay.ipynb)).**

The original posed TRAPI query was this:

```
{
 "message": {
  "query_graph": {
    "edges": {
      "e0": {
        "predicate": "biolink:gene_associated_with_condition",
        "subject": "n0",
        "object": "n1"
      },
      "e1": {
        "predicate":     "biolink:gene_has_variant_that_contributes_to_disease_association",
        "subject": "n0",
        "object": "n2"
      },
      "e2": {
        "subject": "n2",
        "object": "n3",
        "predicate": "biolink:correlated_with"
      }
    },
    "nodes": {
      "n0": {
        "category": "biolink:Gene"
      },
      "n1": {
        "category": "biolink:Disease",
        "id": "MONDO:0007254"
      },
      "n2": {
        "category": "biolink:Disease"
      },
      "n3": {
        "category": "biolink:ChemicalSubstance"
      }
    }
  }
 }
}
```

The original query starts with **"breast cancer" (MONDO:0007254)**.  To reduce the combinatorial explosion, I've replaced that with **"NGLY1-deficiency" (MONDO:0014109)** in the notebook below. The rest of the query path joins that initial disease to a `ChemicalSubstance`, joined by two intermediate nodes with type `Gene` and `Disease`.


**Background**: BioThings Explorer can answer two classes of queries -- "EXPLAIN" and "PREDICT".  EXPLAIN queries are described in [EXPLAIN_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/EXPLAIN_demo.ipynb), and PREDICT queries are described in [PREDICT_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/PREDICT_demo.ipynb). Here, we describe PREDICT queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

## Step 0: Load BioThings Explorer modules

Install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).

In [1]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [2]:
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection
import nest_asyncio
nest_asyncio.apply()

## Step 1: Find representation of the input disease in BTE

In this step, BioThings Explorer translates our query string into a BioThings object, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [3]:
ht = Hint()
input_disease = ht.query("MONDO:0007254")["Disease"][0]

input_disease

{'MONDO': 'MONDO:0007254',
 'DOID': 'DOID:1612',
 'name': 'breast cancer',
 'MESH': 'D001943',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0007254'},
 'display': 'MONDO(MONDO:0007254) DOID(DOID:1612) MESH(D001943) name(breast cancer)',
 'type': 'Disease'}

## Step 2: Find `ChemicalSubstance`s that are associated with the input disease through `Gene` and `Disease` as intermediate nodes

In this section, we find all paths in the knowledge graph that connect the input disease to any entity that is a `ChemicalSubstance` feature.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

In [4]:
fc = FindConnection(input_obj=input_disease, 
                    output_obj='ChemicalSubstance', 
                    intermediate_nodes=['Gene', 'Disease'])

In [5]:
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'breast cancer' and 'ChemicalSubstance'. Paths will have 2 intermediate node.

Intermediate node #1 will have these type constraints: Gene

Intermediate node #2 will have these type constraints: Disease



========== QUERY #1 -- fetch all Gene entities linked to breast cancer ==========

==== Step #1: Query path planning ====

Because breast cancer is of type 'Disease', BTE will query our meta-KG for APIs that can take 'Disease' as input and 'Gene' as output

BTE found 8 apis:

API 1. scigraph(1 API call)
API 2. hetio(1 API call)
API 3. DISEASES(1 API call)
API 4. mgi_gene2phenotype(1 API call)
API 5. biolink(1 API call)
API 6. pharos(1 API call)
API 7. scibite(1 API call)
API 8. cord_disease(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 4.1: https://pending.biothings.io/mgigene2phenotype/query?field

API 8.59: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:10644/diseases?rows=200&direct=true
API 2.5: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=3233
API 2.6: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=27324
API 8.11: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:5174/diseases?rows=200&direct=true
API 2.9: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=27151
API 5.7 semmed_gene failed
API 8.69: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:4214/diseases?rows=200&direct=true
API 8.63: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:126206/diseases?rows=200&direct=true
API 8.66: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1442/diseases?rows=200&direct=true
API 8.67: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:7159/diseases?ro

API 8.92: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:2625/diseases?rows=200&direct=true
API 8.57: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:23038/diseases?rows=200&direct=true
API 10.27: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:5079
API 10.11: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:5174
API 8.61: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:79004/diseases?rows=200&direct=true
API 10.9: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:27151
API 2.37: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=5888
API 8.38: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:8678/diseases?rows=200&direct=true
API 2.38: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=8678
API 8.62: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1511/diseases?rows=200&direct

API 2.75: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=2854
API 8.127: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:9973/diseases?rows=200&direct=true
API 8.128: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:26986/diseases?rows=200&direct=true
API 8.122: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:27086/diseases?rows=200&direct=true
API 8.126: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:55729/diseases?rows=200&direct=true
API 2.76: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=3417
API 2.79: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=57634
API 10.52: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:3921
API 10.53: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:4015
API 2.78: http://ctdbase.org/tools/batchQuery.go?inputType

API 8.146: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1211/diseases?rows=200&direct=true
API 2.110: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=4017
API 2.111: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=406973
API 8.152: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:254122/diseases?rows=200&direct=true
API 2.112: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=3916
API 2.109: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=28908
API 8.149: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:721/diseases?rows=200&direct=true
API 8.151: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1506/diseases?rows=200&direct=true
API 10.98: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:9844
API 2.108: http:/

...

API 4.2462 scibite: No hits
API 4.2463 scibite: No hits
API 4.2464 scibite: No hits
API 4.2465 scibite: No hits
API 4.2466 scibite: No hits
API 4.2467 scibite: No hits
API 4.2468 scibite: No hits
API 4.2469 scibite: No hits
API 4.2470 scibite: No hits
API 4.2471 scibite: No hits
API 4.2472 scibite: No hits
API 4.2473 scibite: No hits
API 4.2474 scibite: No hits
API 4.2475 scibite: No hits
API 4.2476 scibite: No hits
API 4.2477 scibite: No hits
API 4.2478 scibite: No hits
API 4.2479 scibite: No hits
API 4.2480 scibite: No hits
API 4.2481 scibite: No hits
API 4.2482 scibite: No hits
API 4.2483 scibite: No hits
API 4.2484 scibite: No hits
API 4.2485 scibite: No hits
API 4.2486 scibite: No hits
API 4.2487 scibite: No hits
API 4.2488 scibite: No hits
API 4.2489 scibite: No hits
API 4.2490 scibite: No hits
API 4.2491 scibite: No hits
API 4.2492 scibite: No hits
API 4.2493 scibite: No hits
API 4.2494 scibite: No hits
API 4.2495 scibite: No hits
API 4.2496 scibite: No hits
API 4.2497 scibite: 

API 5.514 pharos: No hits
API 5.515 pharos: No hits
API 5.516 pharos: No hits
API 5.517 pharos: No hits
API 5.518 pharos: No hits
API 5.519 pharos: No hits
API 5.520 pharos: No hits
API 5.521 pharos: No hits
API 5.522 pharos: No hits
API 5.523 pharos: No hits
API 5.524 pharos: No hits
API 5.525 pharos: No hits
API 5.526 pharos: No hits
API 5.527 pharos: No hits
API 5.528 pharos: No hits
API 5.529 pharos: No hits
API 5.530 pharos: No hits
API 5.531 pharos: No hits
API 5.532 pharos: No hits
API 5.533 pharos: No hits
API 5.534 pharos: No hits
API 5.535 pharos: No hits
API 5.536 pharos: No hits
API 5.537 pharos: No hits
API 5.538 pharos: No hits
API 5.539 pharos: No hits
API 5.540 pharos: No hits
API 5.541 pharos: No hits
API 5.542 pharos: No hits
API 5.543 pharos: No hits
API 5.544 pharos: No hits
API 5.545 pharos: No hits
API 5.546 pharos: No hits
API 5.547 pharos: No hits
API 5.548 pharos: No hits
API 5.549 pharos: No hits
API 5.550 pharos: No hits
API 5.551 pharos: No hits
API 5.552 ph

API 5.1046 pharos: No hits
API 5.1047 pharos: No hits
API 5.1048 pharos: No hits
API 5.1049 pharos: No hits
API 5.1050 pharos: No hits
API 5.1051 pharos: No hits
API 5.1052 pharos: No hits
API 5.1053 pharos: No hits
API 5.1054 pharos: No hits
API 5.1055 pharos: No hits
API 5.1056 pharos: No hits
API 5.1057 pharos: No hits
API 5.1058 pharos: No hits
API 5.1059 pharos: No hits
API 5.1060 pharos: No hits
API 5.1061 pharos: No hits
API 5.1062 pharos: No hits
API 5.1063 pharos: No hits
API 5.1064 pharos: No hits
API 5.1065 pharos: No hits
API 5.1066 pharos: No hits
API 5.1067 pharos: No hits
API 5.1068 pharos: No hits
API 5.1069 pharos: No hits
API 5.1070 pharos: No hits
API 5.1071 pharos: No hits
API 5.1072 pharos: No hits
API 5.1073 pharos: No hits
API 5.1074 pharos: No hits
API 5.1075 pharos: No hits
API 5.1076 pharos: No hits
API 5.1077 pharos: No hits
API 5.1078 pharos: No hits
API 5.1079 pharos: No hits
API 5.1080 pharos: No hits
API 5.1081 pharos: No hits
API 5.1082 pharos: No hits
A

API 5.1703 pharos: No hits
API 5.1704 pharos: No hits
API 5.1705 pharos: No hits
API 5.1706 pharos: No hits
API 5.1707 pharos: No hits
API 5.1708 pharos: No hits
API 5.1709 pharos: No hits
API 5.1710 pharos: No hits
API 5.1711 pharos: No hits
API 5.1712 pharos: No hits
API 5.1713 pharos: No hits
API 5.1714 pharos: No hits
API 5.1715 pharos: No hits
API 5.1716 pharos: No hits
API 5.1717 pharos: No hits
API 5.1718 pharos: No hits
API 5.1719 pharos: No hits
API 5.1720 pharos: No hits
API 5.1721 pharos: No hits
API 5.1722 pharos: No hits
API 5.1723 pharos: No hits
API 5.1724 pharos: No hits
API 5.1725 pharos: No hits
API 5.1726 pharos: No hits
API 5.1727 pharos: No hits
API 5.1728 pharos: No hits
API 5.1729 pharos: No hits
API 5.1730 pharos: No hits
API 5.1731 pharos: No hits
API 5.1732 pharos: No hits
API 5.1733 pharos: No hits
API 5.1734 pharos: No hits
API 5.1735 pharos: No hits
API 5.1736 pharos: No hits
API 5.1737 pharos: No hits
API 5.1738 pharos: No hits
API 5.1739 pharos: No hits
A

API 5.2530 pharos: No hits
API 5.2531 pharos: No hits
API 5.2532 pharos: No hits
API 5.2533 pharos: No hits
API 5.2534 pharos: No hits
API 5.2535 pharos: No hits
API 5.2536 pharos: No hits
API 5.2537 pharos: No hits
API 5.2538 pharos: No hits
API 5.2539 pharos: No hits
API 5.2540 pharos: No hits
API 5.2541 pharos: No hits
API 5.2542 pharos: No hits
API 5.2543 pharos: No hits
API 5.2544 pharos: No hits
API 5.2545 pharos: No hits
API 5.2546 pharos: No hits
API 5.2547 pharos: No hits
API 5.2548 pharos: No hits
API 5.2549 pharos: No hits
API 5.2550 pharos: No hits
API 5.2551 pharos: No hits
API 5.2552 pharos: No hits
API 5.2553 pharos: No hits
API 5.2554 pharos: No hits
API 5.2555 pharos: No hits
API 5.2556 pharos: No hits
API 5.2557 pharos: No hits
API 5.2558 pharos: No hits
API 5.2559 pharos: No hits
API 5.2560 pharos: No hits
API 5.2561 pharos: No hits
API 5.2562 pharos: No hits
API 5.2563 pharos: No hits
API 5.2564 pharos: No hits
API 5.2565 pharos: No hits
API 5.2566 pharos: No hits
A

## Step 3: Assemble and filter the results

Let's create a `df` object that contains the full output from BioThings Explorer. Each row will show one path that joins the input node to an intermediate node (a `Gene`) to another intermediate node (a `Disease`) to an ending node (a `ChemicalSubstance`). The data frame includes a set of columns with additional details on each node and edge (including human-readable labels, identifiers, and sources). 

In [6]:
df = fc.display_table_view()

In [7]:
df

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,...,node2_type,node2_name,node2_id,pred3,pred3_source,pred3_api,pred3_pubmed,output_type,output_name,output_id
0,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,CD4,NCBIGene:920,related_to,...,Disease,DMD,MONDO:MONDO:0010679,contraindicated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]BIS(N...","name:2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]..."
1,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,CDH17,NCBIGene:1015,related_to,...,Disease,DMD,MONDO:MONDO:0010679,contraindicated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]BIS(N...","name:2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]..."
2,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,CDH17,NCBIGene:1015,related_to,...,Disease,DMD,MONDO:MONDO:0010679,contraindicated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]BIS(N...","name:2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]..."
3,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,SNCA,NCBIGene:6622,related_to,...,Disease,DMD,MONDO:MONDO:0010679,contraindicated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]BIS(N...","name:2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]..."
4,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,TGFB1,NCBIGene:7040,related_to,...,Disease,DMD,MONDO:MONDO:0010679,contraindicated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]BIS(N...","name:2,2'-[(1,4-DIOXOBUTANE-1,4-DIYL)BIS(OXY)]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223609,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,VEGFA,NCBIGene:7422,related_to,...,Disease,ACUTE BRONCHITIS,MESH:D001991,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"(+-)-1-ETHYL-6,8-DIFLUORO-1,4-DIHYDRO-7-(3-MET...","name:(+-)-1-ETHYL-6,8-DIFLUORO-1,4-DIHYDRO-7-(..."
223610,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,HDAC1,NCBIGene:3065,related_to,...,Disease,ACUTE BRONCHITIS,MESH:D001991,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(-)-CEFIXIM,name:(-)-CEFIXIM
223611,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,VEGFA,NCBIGene:7422,related_to,...,Disease,ACUTE BRONCHITIS,MESH:D001991,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(-)-CEFIXIM,name:(-)-CEFIXIM
223612,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,PLAU,NCBIGene:5328,related_to,...,Disease,ANURIA,MONDO:MONDO:0002476,contraindicated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"4'-((4,5-DIHYDRO-2-METHYLIMIDAZO(4,5-D)(1)BENZ...","name:4'-((4,5-DIHYDRO-2-METHYLIMIDAZO(4,5-D)(1..."


Let's do some profiling of the predicates, and then let's filter `pred3` to only include `treated_by` edges.

In [8]:
df.pred1.value_counts()

related_to    209841
Name: pred1, dtype: int64

In [9]:
df.pred2.value_counts()

related_to            209660
contraindicated_by       181
Name: pred2, dtype: int64

In [10]:
df.pred3.value_counts()

contraindicated_by    159517
treated_by             50143
Name: pred3, dtype: int64

In [11]:
dfFilt = df.loc[df['output_name'].notnull()].query('pred3 == "treated_by"')

In [12]:
dfFilt

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,...,node2_type,node2_name,node2_id,pred3,pred3_source,pred3_api,pred3_pubmed,output_type,output_name,output_id
173,BREAST CANCER,Disease,related_to,DISEASE,DISEASES API,None,Gene,ERBB4,NCBIGene:2066,related_to,...,Disease,A-FIB,MESH:D001281,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(+/-)-VERAPAMIL,name:(+/-)-VERAPAMIL
175,BREAST CANCER,Disease,related_to,DISEASE,DISEASES API,None,Gene,ERBB4,NCBIGene:2066,related_to,...,Disease,A-FIB,MESH:D001281,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(+/-)-VERAPAMIL,name:(+/-)-VERAPAMIL
177,BREAST CANCER,Disease,related_to,DISEASE,DISEASES API,None,Gene,ESR2,NCBIGene:2100,related_to,...,Disease,A-FIB,MESH:D001281,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(+/-)-VERAPAMIL,name:(+/-)-VERAPAMIL
178,BREAST CANCER,Disease,related_to,DISEASE,DISEASES API,None,Gene,ESR2,NCBIGene:2100,related_to,...,Disease,A-FIB,MESH:D001281,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(+/-)-VERAPAMIL,name:(+/-)-VERAPAMIL
179,BREAST CANCER,Disease,related_to,DISEASE,DISEASES API,None,Gene,VWF,NCBIGene:7450,related_to,...,Disease,A-FIB,MESH:D001281,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(+/-)-VERAPAMIL,name:(+/-)-VERAPAMIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223607,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,VEGFA,NCBIGene:7422,related_to,...,Disease,ACUTE BRONCHITIS,MESH:D001991,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(1-AMINOCYCLOHEXYL)PENICILLIN,name:(1-AMINOCYCLOHEXYL)PENICILLIN
223608,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,HDAC1,NCBIGene:3065,related_to,...,Disease,ACUTE BRONCHITIS,MESH:D001991,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"(+-)-1-ETHYL-6,8-DIFLUORO-1,4-DIHYDRO-7-(3-MET...","name:(+-)-1-ETHYL-6,8-DIFLUORO-1,4-DIHYDRO-7-(..."
223609,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,VEGFA,NCBIGene:7422,related_to,...,Disease,ACUTE BRONCHITIS,MESH:D001991,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,"(+-)-1-ETHYL-6,8-DIFLUORO-1,4-DIHYDRO-7-(3-MET...","name:(+-)-1-ETHYL-6,8-DIFLUORO-1,4-DIHYDRO-7-(..."
223610,BREAST CANCER,Disease,related_to,Translator Text Mining Provider,CORD Disease API,None,Gene,HDAC1,NCBIGene:3065,related_to,...,Disease,ACUTE BRONCHITIS,MESH:D001991,treated_by,drugcentral,MyChem.info API,NaN,ChemicalSubstance,(-)-CEFIXIM,name:(-)-CEFIXIM


## Step 4: Summarize output `ChemicalSubstance`s

In [13]:
dfFilt.output_name.value_counts().head(50)

(11BETA)-11,17,21-TRIHYDROXYPREGNA-1,4-DIENE-3,20-DIONE                                                                                               631
(11BETA,16ALPHA)-9-FLUORO-11,17,21-TRIHYDROXY-16-METHYLPREGNA-1,4-DIENE-3,20-DIONE                                                                    587
ADRENALEX                                                                                                                                             585
1,2-DEHYDROCORTISONE                                                                                                                                  583
(11BETA)-11,17,21-TRIHYDROXYPREGN-4-ENE-3,20-DIONE                                                                                                    577
11BETA,16ALPHA,17ALPHA,21-TETRAHYDROXY-9ALPHA-FLUORO-1,4-PREGNADIENE-3,20-DIONE                                                                       562
(11BETA,16ALPHA)-9-FLUORO-11,21-DIHYDROXY-16,17-[(1-METHYLETHYLIDENE)BIS(OXY

Suppose we are interested in the links behind `5-METHOXY-N-ACETYLTRYPTAMINE`. Let's examine that subset of paths, specifically printing the frequency of the intermediate `Gene`s and `Disease`s.

In [14]:
dfFilt2 = dfFilt.query('output_name == "5-METHOXY-N-ACETYLTRYPTAMINE"')
dfFilt2

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,...,node2_type,node2_name,node2_id,pred3,pred3_source,pred3_api,pred3_pubmed,output_type,output_name,output_id


In [15]:
dfFilt2.node1_name.value_counts().head(50)

Series([], Name: node1_name, dtype: int64)

In [16]:
dfFilt2.node2_name.value_counts().head(50)

Series([], Name: node2_name, dtype: int64)

## Step 5: export full results to TRAPI

(though this export is not yet TRAPI v1.0 compliant, pending resolution of [this issue](https://github.com/biothings/biothings_explorer/issues/169)...)

In [17]:
fc.to_reasoner_std()

{'query_graph': {'edges': [{'id': 'e0',
    'source_id': 'n0',
    'target_id': 'n1',
    'directed': True},
   {'id': 'e1', 'source_id': 'n1', 'target_id': 'n2', 'directed': True},
   {'id': 'e2', 'source_id': 'n2', 'target_id': 'n3', 'directed': True}],
  'nodes': [{'id': 'n0', 'type': 'Disease', 'curie': 'MONDO:0007254'},
   {'id': 'n1', 'type': 'Gene'},
   {'id': 'n2', 'type': 'Disease'},
   {'id': 'n3', 'type': 'ChemicalSubstance'}]},
 'knowledge_graph': {'nodes': [{'id': 'name:breast cancer',
    'name': 'BREAST CANCER',
    'type': 'Disease',
    'equivalent_identifiers': defaultdict(set,
                {'MONDO': ['MONDO:0007254'],
                 'DOID': ['DOID:1612'],
                 'name': ['BREAST CANCER',
                  'BREAST TUMOR',
                  'CANCER OF BREAST',
                  'MALIGNANT BREAST NEOPLASM',
                  'MALIGNANT BREAST TUMOR',
                  'MALIGNANT NEOPLASM OF BREAST',
                  'MALIGNANT NEOPLASM OF THE BREAST',
  